<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/2%20-%20Overfitting%20e%20Dropout/tecniche_di_regolarizzazione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regolarizzazione

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [50]:
csv_url = "https://raw.githubusercontent.com/ProfAI/tf00/master/datasets/math_class/math_class_deep.csv"
df = pd.read_csv(csv_url)
to_encode = ["sex", "paid", "higher", "internet", "romantic"]
df[to_encode] = pd.get_dummies(df[to_encode], drop_first=True)
df.head()

,student_id,sex,age,traveltime,studytime,failures,Medu,Fedu,paid,higher,internet,romantic,freetime,promoted
0,0,0,18,2,2,0,4,4,0,1,0,0,3,0
1,1,0,17,1,2,0,1,1,0,1,1,0,3,0
2,2,0,15,1,2,3,1,1,1,1,1,0,3,1
3,3,0,15,1,3,0,4,2,1,1,1,1,2,1
4,4,0,16,1,2,0,3,3,1,1,0,0,3,1


In [0]:
X = df.drop(["student_id","promoted"], axis=1).values
y = df["promoted"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Opzione 1: Ridurre la Complessità della Rete

In [0]:
model = keras.models.Sequential([
    keras.layers.Dense(64, input_shape=[X.shape[1]], activation="relu"),
    keras.layers.Dense(1, input_shape=[X.shape[1]], activation="sigmoid"),
])

In [53]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 64)                832       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 65        
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [55]:
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
9/9 [==============================] - 0s 1ms/step - loss: 0.9212 - accuracy: 0.4239
Epoch 2/200
9/9 [==============================] - 0s 1ms/step - loss: 0.6373 - accuracy: 0.6630
Epoch 3/200
9/9 [==============================] - 0s 1ms/step - loss: 0.6456 - accuracy: 0.6775
Epoch 4/200
9/9 [==============================] - 0s 1ms/step - loss: 0.6178 - accuracy: 0.6884
Epoch 5/200
9/9 [==============================] - 0s 1ms/step - loss: 0.6069 - accuracy: 0.6993
Epoch 6/200
9/9 [==============================] - 0s 1ms/step - loss: 0.6015 - accuracy: 0.6920
Epoch 7/200
9/9 [==============================] - 0s 1ms/step - loss: 0.5981 - accuracy: 0.7101
Epoch 8/200
9/9 [==============================] - 0s 1ms/step - loss: 0.5964 - accuracy: 0.6993
Epoch 9/200
9/9 [==============================] - 0s 1ms/step - loss: 0.5930 - accuracy: 0.7246
Epoch 10/200
9/9 [==============================] - 0s 1ms/step - loss: 0.5934 - accuracy: 0.7101
Epoch 11/200
9/9 [===========

In [60]:
print("Loss e Accuracy sul train set: %s " % model.evaluate(X_train, y_train))
print("Loss e Accuracy sul test set: %s " % model.evaluate(X_test, y_test))

9/9 [==============================] - 0s 1ms/step - loss: 0.4844 - accuracy: 0.7572
Loss e Accuracy sul train set: [0.4843565821647644, 0.7572463750839233] 
4/4 [==============================] - 0s 1ms/step - loss: 0.6639 - accuracy: 0.6807
Loss e Accuracy sul test set: [0.6639173030853271, 0.680672287940979] 


## Opzione 2: Utilizzare la Regolarizzazione L1 ed L2

In [0]:
regularizer = keras.regularizers.L1L2(l1=0.01, l2=0.1)

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(128, input_shape=[X.shape[1]], activation="relu"))
model.add(keras.layers.Dense(128, input_shape=[X.shape[1]], activation="relu", kernel_regularizer=regularizer))
model.add(keras.layers.Dense(1, input_shape=[X.shape[1]], activation="sigmoid"))

In [73]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
9/9 [==============================] - 0s 2ms/step - loss: 24.6016 - accuracy: 0.6558
Epoch 2/200
9/9 [==============================] - 0s 1ms/step - loss: 21.3646 - accuracy: 0.6667
Epoch 3/200
9/9 [==============================] - 0s 1ms/step - loss: 18.4723 - accuracy: 0.6703
Epoch 4/200
9/9 [==============================] - 0s 1ms/step - loss: 15.9254 - accuracy: 0.6377
Epoch 5/200
9/9 [==============================] - 0s 1ms/step - loss: 13.6652 - accuracy: 0.6558
Epoch 6/200
9/9 [==============================] - 0s 1ms/step - loss: 11.6867 - accuracy: 0.6667
Epoch 7/200
9/9 [==============================] - 0s 1ms/step - loss: 9.9666 - accuracy: 0.6558
Epoch 8/200
9/9 [==============================] - 0s 2ms/step - loss: 8.4579 - accuracy: 0.7138
Epoch 9/200
9/9 [==============================] - 0s 2ms/step - loss: 7.1649 - accuracy: 0.7029
Epoch 10/200
9/9 [==============================] - 0s 2ms/step - loss: 6.0581 - accuracy: 0.6703
Epoch 11/200
9/9 [=====

In [74]:
print("Loss e Accuracy sul train set: %s " % model.evaluate(X_train, y_train))
print("Loss e Accuracy sul test set: %s " % model.evaluate(X_test, y_test))

9/9 [==============================] - 0s 1ms/step - loss: 0.6014 - accuracy: 0.7319
Loss e Accuracy sul train set: [0.6013984680175781, 0.7318840622901917] 
4/4 [==============================] - 0s 1ms/step - loss: 0.6161 - accuracy: 0.7311
Loss e Accuracy sul test set: [0.6161098480224609, 0.7310924530029297] 


## Opzione 3: Dropout

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(128, input_shape=[X.shape[1]], activation="relu"))
model.add(keras.layers.Dropout(0.7))
model.add(keras.layers.Dense(128, input_shape=[X.shape[1]], activation="relu"))
model.add(keras.layers.Dropout(0.7))
model.add(keras.layers.Dense(1, input_shape=[X.shape[1]], activation="sigmoid"))

In [102]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=300)

Epoch 1/300
9/9 [==============================] - 0s 2ms/step - loss: 1.9989 - accuracy: 0.5543
Epoch 2/300
9/9 [==============================] - 0s 2ms/step - loss: 1.5328 - accuracy: 0.5435
Epoch 3/300
9/9 [==============================] - 0s 2ms/step - loss: 1.5922 - accuracy: 0.4746
Epoch 4/300
9/9 [==============================] - 0s 2ms/step - loss: 1.3124 - accuracy: 0.5254
Epoch 5/300
9/9 [==============================] - 0s 2ms/step - loss: 1.3642 - accuracy: 0.5109
Epoch 6/300
9/9 [==============================] - 0s 2ms/step - loss: 1.0220 - accuracy: 0.6123
Epoch 7/300
9/9 [==============================] - 0s 2ms/step - loss: 1.2159 - accuracy: 0.5326
Epoch 8/300
9/9 [==============================] - 0s 1ms/step - loss: 1.0479 - accuracy: 0.5870
Epoch 9/300
9/9 [==============================] - 0s 1ms/step - loss: 0.9090 - accuracy: 0.5725
Epoch 10/300
9/9 [==============================] - 0s 1ms/step - loss: 0.8834 - accuracy: 0.5797
Epoch 11/300
9/9 [===========

In [103]:
print("Loss e Accuracy sul train set: %s " % model.evaluate(X_train, y_train))
print("Loss e Accuracy sul test set: %s " % model.evaluate(X_test, y_test))

9/9 [==============================] - 0s 1ms/step - loss: 0.4534 - accuracy: 0.7645
Loss e Accuracy sul train set: [0.45337456464767456, 0.7644927501678467] 
4/4 [==============================] - 0s 1ms/step - loss: 0.6478 - accuracy: 0.7059
Loss e Accuracy sul test set: [0.6478139758110046, 0.7058823704719543] 


## Opzione 4: Tecniche Miste

In [0]:
regularizer = keras.regularizers.L1L2(l1=0.01, l2=0.1)

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(64, input_shape=[X.shape[1]], activation="relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(128, input_shape=[X.shape[1]], activation="relu", kernel_regularizer=regularizer))
model.add(keras.layers.Dense(1, input_shape=[X.shape[1]], activation="sigmoid"))

In [119]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
9/9 [==============================] - 0s 2ms/step - loss: 15.7920 - accuracy: 0.6123
Epoch 2/500
9/9 [==============================] - 0s 1ms/step - loss: 13.9765 - accuracy: 0.6449
Epoch 3/500
9/9 [==============================] - 0s 2ms/step - loss: 12.4736 - accuracy: 0.6268
Epoch 4/500
9/9 [==============================] - 0s 2ms/step - loss: 11.0124 - accuracy: 0.6341
Epoch 5/500
9/9 [==============================] - 0s 2ms/step - loss: 9.7693 - accuracy: 0.6703
Epoch 6/500
9/9 [==============================] - 0s 1ms/step - loss: 8.6321 - accuracy: 0.6522
Epoch 7/500
9/9 [==============================] - 0s 2ms/step - loss: 7.6013 - accuracy: 0.6522
Epoch 8/500
9/9 [==============================] - 0s 2ms/step - loss: 6.6836 - accuracy: 0.6739
Epoch 9/500
9/9 [==============================] - 0s 2ms/step - loss: 5.8880 - accuracy: 0.6630
Epoch 10/500
9/9 [==============================] - 0s 2ms/step - loss: 5.2005 - accuracy: 0.6558
Epoch 11/500
9/9 [=======

In [120]:
print("Loss e Accuracy sul train set: %s " % model.evaluate(X_train, y_train))
print("Loss e Accuracy sul test set: %s " % model.evaluate(X_test, y_test))

9/9 [==============================] - 0s 1ms/step - loss: 0.5732 - accuracy: 0.7428
Loss e Accuracy sul train set: [0.5731873512268066, 0.7427536249160767] 
4/4 [==============================] - 0s 1ms/step - loss: 0.6074 - accuracy: 0.7059
Loss e Accuracy sul test set: [0.6073810458183289, 0.7058823704719543] 


## Opzione 5: Raccogliere più dati